# ¿Cuáles son las principales características que llevan a que se ROBEN UNA MOTO O UN VEHÍCULO? 
## Seguro está en los datos

### Data processing step

Let's load all the dataset:

In [11]:
import numpy as np
import pandas as pd
import requests
import json

url = 'https://cali.ckan.io/api/3/action/datastore_search?resource_id=3bbd031c-1642-4d5f-b1b4-7b7d7101320f&q={"MUNICIPIO":"cali"}'
JSONContent = requests.get(url).json()

Finding the column names and the records:

In [12]:
fields = JSONContent['result']['fields']
features = list(map(lambda field: field['id'], fields))
print('Column names: \n', features)

Column names: 
 ['_id', 'DEPARTAMENTO', 'MUNICIPIO', 'CODIGO DANE', 'ARMAS MEDIOS', 'MES', 'DESCRIPCION CONDUCTA', 'GENERO', 'AGRUPA EDAD PERSONA', 'CANTIDAD', 'ANIO']


In [13]:
def getDataframe(array, columnNames):
    df = pd.DataFrame()
    for column in columnNames:
        df[column]=list(map(lambda record: record[column], array))
    return df

In [14]:
records = JSONContent['result']['records']
df = getDataframe(records, features)
tamanio = len(df)
df.head()

,_id,DEPARTAMENTO,MUNICIPIO,CODIGO DANE,ARMAS MEDIOS,MES,DESCRIPCION CONDUCTA,GENERO,AGRUPA EDAD PERSONA,CANTIDAD,ANIO
0,674,VALLE,CALI (CT),76001000,ARMA BLANCA / CORTOPUNZANTE,ENERO,ARTÍCULO 209. ACTOS SEXUALES CON MENOR DE 14 AÑOS,FEMENINO,MENORES,1,2020
1,675,VALLE,CALI (CT),76001000,ARMA BLANCA / CORTOPUNZANTE,ENERO,ARTÍCULO 205. ACCESO CARNAL VIOLENTO,FEMENINO,ADULTOS,2,2020
2,676,VALLE,CALI (CT),76001000,ARMA BLANCA / CORTOPUNZANTE,ENERO,ARTÍCULO 206. ACTO SEXUAL VIOLENTO,FEMENINO,ADOLESCENTES,1,2020
3,680,VALLE,CALI (CT),76001000,ARMA BLANCA / CORTOPUNZANTE,FEBRERO,ARTÍCULO 209. ACTOS SEXUALES CON MENOR DE 14 AÑOS,MASCULINO,MENORES,1,2020
4,681,VALLE,CALI (CT),76001000,ARMA BLANCA / CORTOPUNZANTE,FEBRERO,ARTÍCULO 205. ACCESO CARNAL VIOLENTO,FEMENINO,ADULTOS,2,2020


Due to there are only 100 records, let's proceed to iterate to find the other ones

In [15]:
baseURL='https://cali.ckan.io'
links=JSONContent['result']['_links']
while(tamanio > 0):
    nextURL=JSONContent['result']['_links']['next']
    url = f"{baseURL}{nextURL}"
    JSONContent = requests.get(url).json()
    records = JSONContent['result']['records']
    new_df = getDataframe(records, features)
    tamanio = len(new_df)
    print('TAMANIO:', len(df))
    df = pd.concat([df, new_df])

TAMANIO: 100
TAMANIO: 200
TAMANIO: 227


### Data exploration

Let's explore de data:

In [16]:
fields = JSONContent['result']['fields']
df_info =  pd.DataFrame()
df_info['type'] = list(map(lambda field: field['type'], fields))
df_info['id'] = list(map(lambda field: field['id'], fields))
df_info['label'] = list(map(lambda field: field['info']['label'] if 'info' in field else '', fields))
df_info['note'] = list(map(lambda field: field['info']['notes'] if 'info' in field else '', fields))
df_info

,type,id,label,note
0,int,_id,,
1,text,DEPARTAMENTO,DEPARTAMENTO,Departamento en dónde ocurrió el delíto.
2,text,MUNICIPIO,MUNICIPIO,Municipio en dónde ocurrió el delíto.
3,numeric,CODIGO DANE,CODIGO DANE,Código DANE que identifica al Departamento y M...
4,text,ARMAS MEDIOS,ARMAS MEDIOS,Corresponde al típo de arma o medio usado dura...
5,text,MES,MES,Mes del año en el cúal ocurrió el delíto.
6,text,DESCRIPCION CONDUCTA,DESCRIPCION CONDUCTA,Corresponde al artículo que hace referencia al...
7,text,GENERO,,
8,text,AGRUPA EDAD PERSONA,AGRUPA EDAD PERSONA,Es el grupo de edad al cúal pertenece la perso...
9,numeric,CANTIDAD,CANTIDAD,¿Cantidad de involucrados en el delíto?


In [17]:
df.describe(include=np.object)

,DEPARTAMENTO,MUNICIPIO,ARMAS MEDIOS,MES,DESCRIPCION CONDUCTA,GENERO,AGRUPA EDAD PERSONA
count,227,227,227,227,227,227,227
unique,1,1,6,6,15,2,3
top,VALLE,CALI (CT),SIN EMPLEO DE ARMAS,FEBRERO,ARTÍCULO 209. ACTOS SEXUALES CON MENOR DE 14 AÑOS,FEMENINO,ADULTOS
freq,227,227,93,47,48,176,87


In [18]:
df.to_csv("DELITOS_SEXUALES_2020.csv")